In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bokeh.plotting import figure, show, save
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_file

In [2]:
CSV_PATH = 'C:/Users/hotal/Downloads/nndc_nudat_data_export (1).csv'
OUTPUT_FILE = 'chart_of_nuclides.html'
BINS = pd.IntervalIndex.from_tuples(
    [(-np.inf, 0), (0, 10**-15), (10**-15, 10**-7), (10**-7, 10**-6), (10**-6, 10**-5), (10**-5, 10**-4),
     (10**-4, 10**-3), (10**-3, 10**-2), (10**-2, 10**-1), (10**-1, 10**0), (10**0, 10**1), (10**1, 10**2),
     (10**2, 10**3), (10**3, 10**4), (10**4, 10**5), (10**5, 10**7), (10**7, 10**10), (10**10, 10**15), (10**15, np.inf)
    ])
COLORS = ['#e0e0e0', '#de9473','#f7bdde','#ffc6a5','#ffe7c6','#ffff9c','#ffff10','#e7f784','#d6ef39','#acde63',
          '#52b552','#63bdb5','#63c6de','#00a5c6','#089a94','#0084a5','#3152a5','#29006b','#000000']
TIME_UNITS_TO_SECONDS = {'s':1, 'y':31536000, 'ys':1e-24, 'ms':1e-3, 'ns':1e-9, 'd':86400, 'ps':1e-12, 'm':60,
                         'as':1e-18, 'h':3600, 'us':1e-6, 'mev':1e-100, 'kev':1e-100, 'ev':1e-100, np.nan:1}
MAPPER = {b:c for b,c in zip(BINS,COLORS)}

In [3]:
def load_data():
    return pd.read_csv(CSV_PATH)

def process_data(df):
    return (
        df
        .fillna({"halflife":'UNKNOWN'})
        .assign(hl2=lambda x: x['halflife'].replace({'STABLE':1e100, 'UNKNOWN':-1}).astype(np.float64) * x['halflifeUnit'].map(TIME_UNITS_TO_SECONDS))
        .assign(color = lambda x: pd.cut(x['hl2'], bins = BINS).map(MAPPER))
        .fillna({"halflifeUnit":''})
        .sort_values('hl2', ascending=False)
        .groupby('name').first()
    )

def create_figure(df):
    output_file(OUTPUT_FILE)
    source = ColumnDataSource(df)
    p = figure(width = 900, height = 800)
    r = p.rect(x = 'n', y = 'z', fill_color = 'color', width = 0.99, height = 0.99, source = source, line_color = None)
    hover = HoverTool(renderers = [r], tooltips= [('Name','@name'), ('Protons',"@z"), ('Neutrons','@n'), ('HalfLife','@halflife@halflifeUnit')])
    p.add_tools(hover)
    save(p)
    show(p)

In [4]:
def main():
    df = load_data()
    df = process_data(df)
    create_figure(df)

In [5]:
if __name__ == '__main__':
    main()

In [6]:
%%html
<iframe src="chart_of_nuclides.html", width = 950, height = 850></iframe>